In [3]:
"""
Task 4 - FIXED Proxy Target Variable Engineering
Using business rules instead of K-Means clustering - ALL BUGS FIXED
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import os

print("=" * 70)
print("TASK 4 - FIXED PROXY TARGET VARIABLE ENGINEERING")
print("Using Business Rules Based on EDA Insights")
print("=" * 70)

# Load the data (same as EDA)
df = pd.read_csv("../data/raw/data.csv")
df["TransactionStartTime"] = pd.to_datetime(df["TransactionStartTime"], utc=True)

print(f"Loaded {len(df):,} transactions for {df['CustomerId'].nunique():,} customers")
print(f"Data range: {df['TransactionStartTime'].min()} to {df['TransactionStartTime'].max()}")

# -----------------------------
# 1. Calculate Customer-Level Features
# -----------------------------
print("\n1. CALCULATING CUSTOMER FEATURES")
print("-" * 40)

# Create customer-level features
customer_features = df.groupby('CustomerId').agg({
    'TransactionId': 'count',  # Frequency
    'Amount': ['sum', 'mean', 'std'],  # Monetary
    'Value': 'sum',
    'FraudResult': 'sum',
    'ChannelId': lambda x: (x == 'ChannelId_1').sum(),  # High-risk channel usage
    'TransactionStartTime': ['min', 'max']  # For recency
}).reset_index()

customer_features.columns = [
    'CustomerId', 'transaction_count', 'total_amount', 'avg_amount', 
    'amount_std', 'total_value', 'fraud_count', 'high_risk_channel_use',
    'first_transaction', 'last_transaction'
]

# Convert to timezone-naive for easier calculations
customer_features['first_transaction'] = customer_features['first_transaction'].dt.tz_localize(None)
customer_features['last_transaction'] = customer_features['last_transaction'].dt.tz_localize(None)

# Calculate additional features - FIXED TIMEZONE ISSUE
# Use the max date from data as snapshot date
snapshot_date = pd.Timestamp('2019-02-14')  # Based on EDA max date
customer_features['recency_days'] = (snapshot_date - customer_features['last_transaction']).dt.days
customer_features['customer_tenure_days'] = (customer_features['last_transaction'] - customer_features['first_transaction']).dt.days
customer_features['frequency_per_day'] = customer_features['transaction_count'] / np.maximum(customer_features['customer_tenure_days'], 1)

# Calculate transaction during high-fraud hours (9PM-3AM)
# Convert to local time if needed (assuming UTC, convert to East Africa Time = UTC+3)
df['hour_local'] = (df['TransactionStartTime'] + pd.Timedelta(hours=3)).dt.hour
df['high_fraud_hour'] = ((df['hour_local'] >= 21) | (df['hour_local'] <= 3)).astype(int)

high_fraud_txns = df.groupby('CustomerId')['high_fraud_hour'].mean().reset_index()
high_fraud_txns.columns = ['CustomerId', 'high_fraud_hour_ratio']

customer_features = customer_features.merge(high_fraud_txns, on='CustomerId', how='left')
customer_features['high_fraud_hour_ratio'] = customer_features['high_fraud_hour_ratio'].fillna(0)

print(f"Created {len(customer_features)} customer profiles with {len(customer_features.columns)} features")

# -----------------------------
# 2. Define Business Rules for High-Risk Classification
# -----------------------------
print("\n2. DEFINING BUSINESS RULES FOR HIGH-RISK")
print("-" * 40)

print("Based on EDA Insights:")
print("1. ChannelId_1 has 3.7× higher fraud rate")
print("2. Negative total amount indicates more refunds than purchases")
print("3. Low frequency (< 1 transaction per week) indicates disengagement")
print("4. High proportion of transactions during high-fraud hours (9PM-3AM)")
print("5. High transaction amount variability (unstable spending)")

# Define thresholds
customer_features['is_high_risk'] = 0  # Default to low risk

# Calculate coefficient of variation safely
customer_features['amount_std_filled'] = customer_features['amount_std'].fillna(0)
customer_features['avg_amount_abs'] = np.abs(customer_features['avg_amount']).replace(0, 1)
customer_features['amount_cv'] = customer_features['amount_std_filled'] / customer_features['avg_amount_abs']

# Rule 1: Uses high-risk channel (ChannelId_1)
customer_features.loc[customer_features['high_risk_channel_use'] > 0, 'is_high_risk'] = 1
print(f"  Rule 1 (ChannelId_1 users): {len(customer_features[customer_features['high_risk_channel_use'] > 0])} customers")

# Rule 2: Has negative total amount (more refunds than purchases)
customer_features.loc[customer_features['total_amount'] < 0, 'is_high_risk'] = 1
print(f"  Rule 2 (Negative total): {len(customer_features[customer_features['total_amount'] < 0])} customers")

# Rule 3: Low frequency (less than 1 transaction per week)
customer_features.loc[customer_features['frequency_per_day'] < 1/7, 'is_high_risk'] = 1
print(f"  Rule 3 (Low frequency): {len(customer_features[customer_features['frequency_per_day'] < 1/7])} customers")

# Rule 4: High proportion of transactions during high-fraud hours
customer_features.loc[customer_features['high_fraud_hour_ratio'] > 0.3, 'is_high_risk'] = 1  # Lowered to 30%
print(f"  Rule 4 (High fraud hour ratio): {len(customer_features[customer_features['high_fraud_hour_ratio'] > 0.3])} customers")

# Rule 5: High transaction variability (more than 50% CV)
customer_features.loc[customer_features['amount_cv'] > 0.5, 'is_high_risk'] = 1
print(f"  Rule 5 (High variability): {len(customer_features[customer_features['amount_cv'] > 0.5])} customers")

# Optional: Has any fraud history
customer_features.loc[customer_features['fraud_count'] > 0, 'is_high_risk'] = 1
print(f"  Rule 6 (Fraud history): {len(customer_features[customer_features['fraud_count'] > 0])} customers")

print("\nApplied 6 business rules to identify high-risk customers")

# -----------------------------
# 3. Analyze Results
# -----------------------------
print("\n3. ANALYZING RISK DISTRIBUTION")
print("-" * 40)

risk_counts = customer_features['is_high_risk'].value_counts()
risk_percentage = risk_counts / len(customer_features) * 100

print(f"Risk distribution:")
print(f"  Low Risk: {risk_counts.get(0, 0):,} customers ({risk_percentage.get(0, 0):.1f}%)")
print(f"  High Risk: {risk_counts.get(1, 0):,} customers ({risk_percentage.get(1, 0):.1f}%)")

# Check which rules triggered most (excluding those already marked high risk by previous rules)
print(f"\nNumber of customers triggering each rule:")
rules_summary = {
    'Uses ChannelId_1': (customer_features['high_risk_channel_use'] > 0).sum(),
    'Negative total amount': (customer_features['total_amount'] < 0).sum(),
    'Low frequency (<1/week)': (customer_features['frequency_per_day'] < 1/7).sum(),
    'High fraud hour ratio (>30%)': (customer_features['high_fraud_hour_ratio'] > 0.3).sum(),
    'High variability (CV>50%)': (customer_features['amount_cv'] > 0.5).sum(),
    'Has fraud history': (customer_features['fraud_count'] > 0).sum()
}

for rule, count in rules_summary.items():
    print(f"  {rule:30}: {count:5d} ({count/len(customer_features)*100:.1f}%)")

# -----------------------------
# 4. Visualize High-Risk Characteristics
# -----------------------------
print("\n4. CHARACTERISTICS OF HIGH-RISK CUSTOMERS")
print("-" * 40)

low_risk = customer_features[customer_features['is_high_risk'] == 0]
high_risk = customer_features[customer_features['is_high_risk'] == 1]

print(f"\nComparing high-risk vs low-risk customers:")
print(f"{'Metric':30} {'Low Risk':>15} {'High Risk':>15} {'Diff %':>10}")
print("-" * 80)

comparison_data = []
for col in ['transaction_count', 'total_amount', 'avg_amount', 'recency_days', 
            'frequency_per_day', 'high_fraud_hour_ratio', 'amount_cv']:
    if col in customer_features.columns:
        low_mean = low_risk[col].mean()
        high_mean = high_risk[col].mean()
        
        # FIXED: Handle division by zero properly
        if abs(low_mean) < 0.0001:  # Near zero
            diff_pct = 100 if high_mean > 0 else -100 if high_mean < 0 else 0
        else:
            diff_pct = ((high_mean - low_mean) / abs(low_mean)) * 100
        
        print(f"{col:30} {low_mean:15.2f} {high_mean:15.2f} {diff_pct:10.1f}%")
        comparison_data.append({
            'metric': col,
            'low_risk': low_mean,
            'high_risk': high_mean,
            'diff_pct': diff_pct
        })

# -----------------------------
# 5. Prepare for Model Training
# -----------------------------
print("\n5. PREPARING DATA FOR MODEL TRAINING")
print("-" * 40)

# Select features for modeling
feature_columns = [
    'transaction_count', 'total_amount', 'avg_amount', 'amount_std',
    'recency_days', 'customer_tenure_days', 'frequency_per_day',
    'high_risk_channel_use', 'high_fraud_hour_ratio', 'amount_cv'
]

# Create model-ready dataset
model_data = customer_features[['CustomerId', 'is_high_risk'] + feature_columns].copy()

# Handle missing values
for col in feature_columns:
    if model_data[col].isnull().any():
        model_data[col] = model_data[col].fillna(model_data[col].median())

print(f"Model data shape: {model_data.shape}")
print(f"Features: {', '.join(feature_columns[:5])}...")
print(f"Target: is_high_risk (1 = high risk, 0 = low risk)")

# Check class distribution
class_dist = model_data['is_high_risk'].value_counts(normalize=True)
print(f"\nClass distribution:")
for risk_class, percentage in class_dist.items():
    label = "High Risk" if risk_class == 1 else "Low Risk"
    print(f"  {label}: {percentage:.1%}")

# -----------------------------
# 6. Save Results
# -----------------------------
print("\n6. SAVING RESULTS")
print("-" * 40)

# Create directories
os.makedirs('data/processed', exist_ok=True)

# Save processed data
output_path = 'data/processed/customers_with_proxy_target_fixed.csv'
model_data.to_csv(output_path, index=False)
print(f"✓ Saved to {output_path}")

# Save target separately
target_path = 'data/processed/proxy_target_fixed.csv'
model_data[['CustomerId', 'is_high_risk']].to_csv(target_path, index=False)
print(f"✓ Saved target to {target_path}")

# Save feature list
features_path = 'data/processed/model_features_fixed.txt'
with open(features_path, 'w') as f:
    for feature in feature_columns:
        f.write(f"{feature}\n")
print(f"✓ Saved feature list to {features_path}")

# Save descriptive statistics
stats_path = 'data/processed/feature_statistics.csv'
stats_df = model_data[feature_columns].describe().T
stats_df.to_csv(stats_path)
print(f"✓ Saved feature statistics to {stats_path}")

# Save comparison data
comparison_df = pd.DataFrame(comparison_data)
comparison_path = 'data/processed/risk_comparison.csv'
comparison_df.to_csv(comparison_path, index=False)
print(f"✓ Saved risk comparison data to {comparison_path}")

# -----------------------------
# 7. Summary Report
# -----------------------------
print("\n" + "=" * 70)
print("PROXY TARGET ENGINEERING COMPLETE - SUMMARY")
print("=" * 70)

print(f"\n📊 RISK DISTRIBUTION")
print(f"   Total customers: {len(model_data):,}")
print(f"   Low risk: {risk_counts.get(0, 0):,} ({risk_percentage.get(0, 0):.1f}%)")
print(f"   High risk: {risk_counts.get(1, 0):,} ({risk_percentage.get(1, 0):.1f}%)")

print(f"\n📈 CLASS BALANCE")
imbalance_ratio = risk_counts.get(1, 0) / risk_counts.get(0, 0) if risk_counts.get(0, 0) > 0 else 1
print(f"   Ratio (High:Low): 1:{1/imbalance_ratio:.1f}")
if 0.25 <= imbalance_ratio <= 0.75:
    print(f"   ✅ Good class balance for modeling")
else:
    print(f"   ⚠ Class imbalance - consider sampling techniques during training")

print(f"\n🔧 BUSINESS RULES APPLIED")
rules_list = [
    "Uses ChannelId_1 (high-risk channel)",
    "Has negative total amount",
    "Frequency < 1 transaction/week",
    ">30% transactions in high-fraud hours (9PM-3AM local time)",
    "High transaction variability (CV > 50%)",
    "Any fraud history"
]
for i, rule in enumerate(rules_list, 1):
    print(f"   {i}. {rule}")

print(f"\n🎯 FEATURES FOR MODELING")
print(f"   Number of features: {len(feature_columns)}")
print(f"   Key behavioral features:")
print(f"     • Frequency: transaction_count, frequency_per_day")
print(f"     • Monetary: total_amount, avg_amount")
print(f"     • Recency: recency_days, customer_tenure_days")
print(f"     • Risk indicators: high_risk_channel_use, high_fraud_hour_ratio, amount_cv")

print(f"\n💾 OUTPUT FILES")
files = [
    (output_path, "Complete dataset for modeling (CustomerId + features + target)"),
    (target_path, "Target variable only (CustomerId + is_high_risk)"),
    (features_path, "List of features for modeling"),
    (stats_path, "Feature descriptive statistics"),
    (comparison_path, "High-risk vs low-risk comparison")
]
for path, desc in files:
    if os.path.exists(path):
        print(f"   ✓ {path}")
        print(f"     {desc}")

print(f"\n" + "=" * 70)
print("READY FOR TASK 5 - MODEL TRAINING")
print("=" * 70)
print(f"\nNext steps:")
print(f"   1. Load 'customers_with_proxy_target_fixed.csv'")
print(f"   2. Split into train/test sets (80/20)")
print(f"   3. Train at least 2 models (e.g., Logistic Regression, Random Forest)")
print(f"   4. Hyperparameter tuning with GridSearchCV")
print(f"   5. Evaluate using: Accuracy, Precision, Recall, F1, ROC-AUC")
print(f"   6. Track experiments with MLflow")
print(f"\nExample code for Task 5:")
print(f"   import pandas as pd")
print(f"   from sklearn.model_selection import train_test_split")
print(f"   ")
print(f"   data = pd.read_csv('data/processed/customers_with_proxy_target_fixed.csv')")
print(f"   X = data.drop(['CustomerId', 'is_high_risk'], axis=1)")
print(f"   y = data['is_high_risk']")
print(f"   ")
print(f"   X_train, X_test, y_train, y_test = train_test_split(")
print(f"       X, y, test_size=0.2, random_state=42, stratify=y")
print(f"   )")

print(f"\n✅ Task 4 is now complete and ready for model training!")

TASK 4 - FIXED PROXY TARGET VARIABLE ENGINEERING
Using Business Rules Based on EDA Insights
Loaded 95,662 transactions for 3,742 customers
Data range: 2018-11-15 02:18:49+00:00 to 2019-02-13 10:01:28+00:00

1. CALCULATING CUSTOMER FEATURES
----------------------------------------
Created 3742 customer profiles with 14 features

2. DEFINING BUSINESS RULES FOR HIGH-RISK
----------------------------------------
Based on EDA Insights:
1. ChannelId_1 has 3.7× higher fraud rate
2. Negative total amount indicates more refunds than purchases
3. Low frequency (< 1 transaction per week) indicates disengagement
4. High proportion of transactions during high-fraud hours (9PM-3AM)
5. High transaction amount variability (unstable spending)
  Rule 1 (ChannelId_1 users): 92 customers
  Rule 2 (Negative total): 192 customers
  Rule 3 (Low frequency): 71 customers
  Rule 4 (High fraud hour ratio): 904 customers
  Rule 5 (High variability): 2797 customers
  Rule 6 (Fraud history): 54 customers

Applied 6